In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#import Libraries
import findspark
findspark.init('./spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
import pyspark.sql.functions as F

In [3]:
# sc = SparkContext()
conf = SparkConf().setAppName("pySparkMerge").setMaster("local")
sc = SparkContext.getOrCreate(conf=conf)

In [4]:
sc.version
sc.pythonVer

'2.1.1'

'3.7'

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import pickle
import seaborn as sns
sns.set()

In [6]:
##CREATING "SPARKSESSION" from existing "SPARKCONTEXT"
sqlContext = SQLContext(sc)
spark = sqlContext.sparkSession

In [7]:
c1 = spark.read.csv("./KaggleV2-May-2016-d1.csv", header=True)
print(c1.show(2))
print((c1.count(), len(c1.columns)))
print(c1.select(F.countDistinct("PatientId")).show())

+-----------+-----------+------------+--------+----------+-------+
|  PatientId|Scholarship|Hipertension|Diabetes|Alcoholism|Handcap|
+-----------+-----------+------------+--------+----------+-------+
|2.98725E+13|          0|           1|       0|         0|      0|
|5.58998E+14|          0|           0|       0|         0|      0|
+-----------+-----------+------------+--------+----------+-------+
only showing top 2 rows

None
(110527, 6)
+-------------------------+
|count(DISTINCT PatientId)|
+-------------------------+
|                    61744|
+-------------------------+

None


In [8]:
c2 = spark.read.csv("./KaggleV2-May-2016-d2.csv", header=True)
print(c2.show(2))
print((c2.count(), len(c2.columns)))

# print(c2.select("PatientId").distinct().show())
print(c2.select(F.countDistinct("PatientId")).show())

+-----------+-------------+------+--------------------+--------------------+---+---------------+------------+-------+
|  PatientId|AppointmentID|Gender|        ScheduledDay|      AppointmentDay|Age|  Neighbourhood|SMS_received|No-show|
+-----------+-------------+------+--------------------+--------------------+---+---------------+------------+-------+
|2.98725E+13|      5642903|     F|2016-04-29T18:38:08Z|2016-04-29T00:00:00Z| 62|JARDIM DA PENHA|           0|     No|
|5.58998E+14|      5642503|     M|2016-04-29T16:08:27Z|2016-04-29T00:00:00Z| 56|JARDIM DA PENHA|           0|     No|
+-----------+-------------+------+--------------------+--------------------+---+---------------+------------+-------+
only showing top 2 rows

None
(110527, 9)
+-------------------------+
|count(DISTINCT PatientId)|
+-------------------------+
|                    61744|
+-------------------------+

None


In [9]:
###rename the column
c1 = c1.withColumnRenamed('PatientId', 'PatientId_x')
c2 = c2.withColumnRenamed('PatientId', 'PatientId_y')

In [10]:
import pyspark.sql.functions as sparkf
from pyspark.sql.functions import col

# This will return a new DF with all the columns + id

c1 = c1.withColumn('id', sparkf.monotonically_increasing_id())
c2 = c2.withColumn('id', sparkf.monotonically_increasing_id())

## MERGE FILES FROM DIFFERENT SOURCES
# Same index from both file
merged = c1.join(c2, c1.id == c2.id, 'left').drop(c2.id)
merged.show(4)
print(merged.count(), len(merged.columns))

+-----------+-----------+------------+--------+----------+-------+---+-----------+-------------+------+--------------------+--------------------+---+-----------------+------------+-------+---+
|PatientId_x|Scholarship|Hipertension|Diabetes|Alcoholism|Handcap| id|PatientId_y|AppointmentID|Gender|        ScheduledDay|      AppointmentDay|Age|    Neighbourhood|SMS_received|No-show| id|
+-----------+-----------+------------+--------+----------+-------+---+-----------+-------------+------+--------------------+--------------------+---+-----------------+------------+-------+---+
|2.98725E+13|          0|           1|       0|         0|      0|  0|2.98725E+13|      5642903|     F|2016-04-29T18:38:08Z|2016-04-29T00:00:00Z| 62|  JARDIM DA PENHA|           0|     No|  0|
|5.58998E+14|          0|           0|       0|         0|      0|  1|5.58998E+14|      5642503|     M|2016-04-29T16:08:27Z|2016-04-29T00:00:00Z| 56|  JARDIM DA PENHA|           0|     No|  1|
|4.26296E+12|          0|          

In [11]:
# Drop the splits column
merged = merged.drop('PatientId_y')
merged = merged.drop('id')
merged = merged.withColumnRenamed('PatientId_x', 'PatientId')
merged.columns

['PatientId',
 'Scholarship',
 'Hipertension',
 'Diabetes',
 'Alcoholism',
 'Handcap',
 'AppointmentID',
 'Gender',
 'ScheduledDay',
 'AppointmentDay',
 'Age',
 'Neighbourhood',
 'SMS_received',
 'No-show']

In [12]:
merged.write.csv('./KaggleV2-chart.csv', header=True)

In [13]:
df0 = spark.read.csv("./KaggleV2-chart.csv/", header=True)

print(df0.count(), )
print((df0.count(), len(df0.columns)))

df0.columns

110527
(110527, 14)


['PatientId',
 'Scholarship',
 'Hipertension',
 'Diabetes',
 'Alcoholism',
 'Handcap',
 'AppointmentID',
 'Gender',
 'ScheduledDay',
 'AppointmentDay',
 'Age',
 'Neighbourhood',
 'SMS_received',
 'No-show']

In [14]:
## INITIAL VIEW ON DATA ## descripe in pyspark
summary1A = df0.describe().show()
print(summary1A)
print(df0.printSchema())
# print(df0.summary().show())

+-------+--------------------+-------------------+-------------------+-------------------+--------------------+--------------------+-----------------+------+--------------------+--------------------+------------------+-------------+-------------------+-------+
|summary|           PatientId|        Scholarship|       Hipertension|           Diabetes|          Alcoholism|             Handcap|    AppointmentID|Gender|        ScheduledDay|      AppointmentDay|               Age|Neighbourhood|       SMS_received|No-show|
+-------+--------------------+-------------------+-------------------+-------------------+--------------------+--------------------+-----------------+------+--------------------+--------------------+------------------+-------------+-------------------+-------+
|  count|              110527|             110527|             110527|             110527|              110527|              110527|           110527|110527|              110527|              110527|            110527

In [15]:
## CHECK ANY MISSING VALUSE
### Get count of both null and missing values in pyspark
from pyspark.sql.functions import isnan, when, count, col
print( df0.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df0.columns]).show() )

# No. of missing value in the 'SMS_received'
df0.where(col('SMS_received').isNull()).count()

# Percentage of missing value in the 'SMS_received'
df0.where(col('SMS_received').isNull()).count()/df0.count()*100

+---------+-----------+------------+--------+----------+-------+-------------+------+------------+--------------+---+-------------+------------+-------+
|PatientId|Scholarship|Hipertension|Diabetes|Alcoholism|Handcap|AppointmentID|Gender|ScheduledDay|AppointmentDay|Age|Neighbourhood|SMS_received|No-show|
+---------+-----------+------------+--------+----------+-------+-------------+------+------------+--------------+---+-------------+------------+-------+
|        0|          0|           0|       0|         0|      0|            0|     0|           0|             0|  0|            0|          59|      0|
+---------+-----------+------------+--------+----------+-------+-------------+------+------------+--------------+---+-------------+------------+-------+

None


59

0.05338062192948329

In [16]:
# df0.printSchema()
df0.show(2)
from pyspark.sql.types import IntegerType
df0 = df0.withColumn("Scholarship", df0["Scholarship"].cast(IntegerType()))
df0 = df0.withColumn("Hipertension", df0["Hipertension"].cast(IntegerType()))
df0 = df0.withColumn("Diabetes", df0["Diabetes"].cast(IntegerType()))
df0 = df0.withColumn("Alcoholism", df0["Alcoholism"].cast(IntegerType()))
df0 = df0.withColumn("Handcap", df0["Handcap"].cast(IntegerType()))
df0 = df0.withColumn("AppointmentID", df0["AppointmentID"].cast(IntegerType()))
df0 = df0.withColumn("Age", df0["Age"].cast(IntegerType()))
df0 = df0.withColumn("SMS_received", df0["SMS_received"].cast(IntegerType()))
df0.printSchema()

+-----------+-----------+------------+--------+----------+-------+-------------+------+--------------------+--------------------+---+---------------+------------+-------+
|  PatientId|Scholarship|Hipertension|Diabetes|Alcoholism|Handcap|AppointmentID|Gender|        ScheduledDay|      AppointmentDay|Age|  Neighbourhood|SMS_received|No-show|
+-----------+-----------+------------+--------+----------+-------+-------------+------+--------------------+--------------------+---+---------------+------------+-------+
|2.98725E+13|          0|           1|       0|         0|      0|      5642903|     F|2016-04-29T18:38:08Z|2016-04-29T00:00:00Z| 62|JARDIM DA PENHA|           0|     No|
|5.58998E+14|          0|           0|       0|         0|      0|      5642503|     M|2016-04-29T16:08:27Z|2016-04-29T00:00:00Z| 56|JARDIM DA PENHA|           0|     No|
+-----------+-----------+------------+--------+----------+-------+-------------+------+--------------------+--------------------+---+------------

In [17]:
df0['PatientId','Hipertension'].show()

+-----------+------------+
|  PatientId|Hipertension|
+-----------+------------+
|2.98725E+13|           1|
|5.58998E+14|           0|
|4.26296E+12|           0|
|8.67951E+11|           0|
|8.84119E+12|           1|
|9.59851E+13|           1|
|7.33688E+14|           0|
|3.44983E+12|           0|
|5.63947E+13|           0|
|7.81246E+13|           0|
|7.34536E+14|           0|
|7.54295E+12|           0|
|5.66655E+14|           0|
|9.11395E+14|           0|
|9.98847E+13|           0|
|99948393975|           0|
|8.45744E+13|           0|
| 1.4795E+13|           0|
|1.71354E+13|           0|
|7.22329E+12|           0|
+-----------+------------+
only showing top 20 rows



In [18]:
df0 = df0.toPandas()

In [19]:
print(df0)

          PatientId  Scholarship  Hipertension  Diabetes  Alcoholism  Handcap  \
0       2.98725E+13            0             1         0           0        0   
1       5.58998E+14            0             0         0           0        0   
2       4.26296E+12            0             0         0           0        0   
3       8.67951E+11            0             0         0           0        0   
4       8.84119E+12            0             1         1           0        0   
5       9.59851E+13            0             1         0           0        0   
6       7.33688E+14            0             0         0           0        0   
7       3.44983E+12            0             0         0           0        0   
8       5.63947E+13            0             0         0           0        0   
9       7.81246E+13            0             0         0           0        0   
10      7.34536E+14            0             0         0           0        0   
11      7.54295E+12         

In [20]:
#Variable Summary - Gender
summary1C = df0.groupby('Gender').size()
summary1C
levels = np.sort(df0['Gender'].unique()) 
df0.Gender.value_counts()[levels].plot(kind='bar')

Gender
F    71840
M    38687
dtype: int64

In [21]:
#Variable Summary - Others
df0.hist(figsize=(12,10))

summary1D = df0.groupby('PatientId').size()
summary1D

summary1D = df0.groupby('AppointmentID').size()
summary1D

summary1E = df0.groupby('ScheduledDay').size()
summary1E

summary1F = df0.groupby('AppointmentDay').size()
summary1F

summary1G = df0.groupby('Neighbourhood').size()
summary1G

summary1I = df0.groupby('Age').size()
summary1I

summary1J = df0.groupby('Scholarship').size()
summary1J

summary1K = df0.groupby('Hipertension').size()
summary1K

summary1L = df0.groupby('Diabetes').size()
summary1L

summary1M = df0.groupby('Alcoholism').size()
summary1M

summary1N = df0.groupby('Handcap').size()
summary1N

summary1O = df0.groupby('SMS_received').size()
summary1O

summary1P = df0.groupby('No-show').size()
summary1P
plt.hist(df0['No-show'], bins=len(df0['No-show'].unique()),rwidth=0.5)

PatientId
1.11115E+13     1
1.11125E+14     2
1.11145E+13     1
1.11162E+13     1
1.11163E+12     3
1.11185E+12     1
1.11196E+12     1
1.11238E+13     1
1.11266E+13     1
1.11277E+12     1
1.11285E+12     1
1.11296E+14     1
1.11311E+11     2
1.11316E+12     1
1.11321E+13     1
1.11345E+14     1
1.11346E+14     2
1.11368E+14     1
1.11374E+14     1
1.11416E+14     4
1.1143E+12      1
1.11456E+13     1
1.11463E+13     2
1.11469E+14     1
1.11523E+14     1
1.11525E+12     1
1.11525E+13     1
1.11526E+12     4
1.11536E+11     1
1.11547E+11     1
               ..
99512431        1
995277946       2
99575164442     1
99612449681     1
99612996974     1
9961417576      1
99637671331    84
99637837716     1
99638668487     2
99676396336     5
99679785292     2
99682621349     1
99686781928     1
99716715215     4
99718131773     2
99728164173     2
997421611       1
99761723841     2
997694529       3
99777928223     1
99813573358     2
99821629648     2
99834588263     1
99838843772     1


AppointmentID
5030230    1
5122866    1
5134197    1
5134220    1
5134223    1
5134224    1
5134227    1
5134228    1
5134233    1
5134239    1
5134243    1
5134249    1
5139826    1
5139831    1
5139832    1
5139834    1
5139839    1
5139848    1
5139853    1
5139857    1
5139859    1
5139864    1
5140065    1
5140069    1
5140074    1
5140081    1
5140090    1
5140100    1
5140105    1
5140112    1
          ..
5790245    1
5790248    1
5790275    1
5790284    1
5790287    1
5790292    1
5790319    1
5790322    1
5790330    1
5790351    1
5790352    1
5790355    1
5790358    1
5790362    1
5790364    1
5790367    1
5790369    1
5790370    1
5790372    1
5790376    1
5790377    1
5790416    1
5790434    1
5790447    1
5790452    1
5790461    1
5790464    1
5790466    1
5790481    1
5790484    1
Length: 110527, dtype: int64

ScheduledDay
2015-11-10T07:13:56Z    1
2015-12-03T08:17:28Z    1
2015-12-07T10:40:59Z    1
2015-12-07T10:42:42Z    1
2015-12-07T10:43:01Z    1
2015-12-07T10:43:17Z    1
2015-12-07T10:43:34Z    1
2015-12-07T10:43:50Z    1
2015-12-07T10:44:07Z    1
2015-12-07T10:44:25Z    1
2015-12-07T10:44:40Z    1
2015-12-07T10:45:01Z    1
2015-12-08T13:30:21Z    1
2015-12-08T13:30:41Z    1
2015-12-08T13:31:04Z    1
2015-12-08T13:31:21Z    1
2015-12-08T13:31:45Z    1
2015-12-08T13:32:14Z    1
2015-12-08T13:32:34Z    1
2015-12-08T13:33:09Z    1
2015-12-08T13:33:28Z    1
2015-12-08T13:33:50Z    1
2015-12-08T13:58:50Z    1
2015-12-08T13:59:33Z    1
2015-12-08T14:00:52Z    1
2015-12-08T14:01:28Z    1
2015-12-08T14:02:04Z    1
2015-12-08T14:02:31Z    1
2015-12-08T14:03:00Z    1
2015-12-08T14:03:23Z    1
                       ..
2016-06-08T17:23:59Z    1
2016-06-08T17:24:34Z    1
2016-06-08T17:37:08Z    1
2016-06-08T17:42:29Z    1
2016-06-08T17:45:18Z    1
2016-06-08T17:50:25Z    1
2016-06-08T18:07:30Z    1

AppointmentDay
2016-04-29T00:00:00Z    3235
2016-05-02T00:00:00Z    4376
2016-05-03T00:00:00Z    4256
2016-05-04T00:00:00Z    4168
2016-05-05T00:00:00Z    4273
2016-05-06T00:00:00Z    3879
2016-05-09T00:00:00Z    4520
2016-05-10T00:00:00Z    4308
2016-05-11T00:00:00Z    4474
2016-05-12T00:00:00Z    4394
2016-05-13T00:00:00Z    3987
2016-05-14T00:00:00Z      39
2016-05-16T00:00:00Z    4613
2016-05-17T00:00:00Z    4372
2016-05-18T00:00:00Z    4373
2016-05-19T00:00:00Z    4270
2016-05-20T00:00:00Z    3828
2016-05-24T00:00:00Z    4009
2016-05-25T00:00:00Z    3909
2016-05-30T00:00:00Z    4514
2016-05-31T00:00:00Z    4279
2016-06-01T00:00:00Z    4464
2016-06-02T00:00:00Z    4310
2016-06-03T00:00:00Z    4090
2016-06-06T00:00:00Z    4692
2016-06-07T00:00:00Z    4416
2016-06-08T00:00:00Z    4479
dtype: int64

Neighbourhood
AEROPORTO                 8
ANDORINHAS             2262
ANT?NIO HON?RIO         271
ARIOVALDO FAVALESSA     282
BARRO VERMELHO          423
BELA VISTA             1907
BENTO FERREIRA          858
BOA VISTA               312
BONFIM                 2773
CARATO?RA              2565
CENTRO                 3334
COMDUSA                 310
CONQUISTA               849
CONSOLA??O             1376
CRUZAMENTO             1398
DA PENHA               2217
DE LOURDES              305
DO CABRAL               560
DO MOSCOSO              413
DO QUADRO               849
ENSEADA DO SU?          235
ESTRELINHA              538
FONTE GRANDE            682
FORTE S?O JO?O         1889
FRADINHOS               258
GOIABEIRAS              700
GRANDE VIT?RIA         1071
GURIGICA               2018
HORTO                   175
ILHA DAS CAIEIRAS      1071
                       ... 
PARQUE INDUSTRIAL         1
PARQUE MOSCOSO          802
PIEDADE                 452
PONTAL DE CAMBURI        69
PRAIA 

...

Diabetes
0    102584
1      7943
dtype: int64

Alcoholism
0    107167
1      3360
dtype: int64

Handcap
0    108286
1      2042
2       183
3        13
4         3
dtype: int64

SMS_received
0.0    75005
1.0    35463
dtype: int64

No-show
No     88208
Yes    22319
dtype: int64

In [22]:
## Count Plot to see the distribution of individual variable and 'No-show' variable
sns.countplot('PatientId', data=df0, hue='No-show')
sns.countplot('Scholarship', data=df0, hue='No-show')
sns.countplot('Hipertension', data=df0, hue='No-show')
sns.countplot('Diabetes', data=df0, hue='No-show')
sns.countplot('Alcoholism', data=df0, hue='No-show')
sns.countplot('Handcap', data=df0, hue='No-show')
sns.countplot('Gender', data=df0, hue='No-show')
sns.countplot('AppointmentDay', data=df0, hue='No-show')
sns.countplot('Age', data=df0, hue='No-show')
sns.countplot('Neighbourhood', data=df0, hue='No-show')
sns.countplot('SMS_received', data=df0, hue='No-show')

In [23]:
## Violin Plot to see the categorical data 
sns.catplot(data=df0, kind="violin", x="Gender", y="Hipertension", hue="No-show", split=True)
sns.catplot(data=df0, kind="violin", x="Gender", y="Diabetes", hue="No-show", split=True)#
sns.catplot(data=df0, kind="violin", x="Gender", y="Alcoholism", hue="No-show", split=True)
sns.catplot(data=df0, kind="violin", x="Gender", y="Handcap", hue="No-show", split=True)
sns.catplot(data=df0, kind="violin", x="Gender", y="AppointmentID", hue="No-show", split=True)
sns.catplot(data=df0, kind="violin", x="Gender", y="Age", hue="No-show", split=True)
sns.catplot(data=df0, kind="violin", x="Gender", y="SMS_received", hue="No-show", split=True)

sns.catplot(data=df0, kind="violin", x="SMS_received", y="Hipertension", hue="No-show", split=True)
sns.catplot(data=df0, kind="violin", x="SMS_received", y="Diabetes", hue="No-show", split=True)
sns.catplot(data=df0, kind="violin", x="SMS_received", y="Alcoholism", hue="No-show", split=True)
sns.catplot(data=df0, kind="violin", x="SMS_received", y="Handcap", hue="No-show", split=True)
sns.catplot(data=df0, kind="violin", x="SMS_received", y="AppointmentID", hue="No-show", split=True)
sns.catplot(data=df0, kind="violin", x="SMS_received", y="Age", hue="No-show", split=True)
